In [ ]:
#update scikit-learn
!pip uninstall scikit-learn -y
!pip install scikit-learn
#!pip install scikit-multilearn
!pip install iterative-stratification
#!pip install sklearn-contrib-lightning
!pip install --upgrade numpy

!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

Found existing installation: scikit-learn 1.0.1
Uninstalling scikit-learn-1.0.1:
  Successfully uninstalled scikit-learn-1.0.1
     |████████████████████████████████| 23.2 MB 51.1 MB/s 
     |████████████████████████████████| 15.7 MB 5.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.4 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 4.06 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 131 kB 5.3 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 8.4 MB 44.0 MB/s 
     |████████████████████████████████| 138 kB 49.1 MB/s 
     |████████████████████████████████| 127 kB 48.5 MB/s 
  Created wheel for bert-sklearn: filename

In [ ]:
import os
os.chdir("bert-sklearn")
print(os.listdir())

['bert_sklearn', 'tests', 'LICENSE', 'demo.ipynb', 'Options.md', 'setup.py', 'glue_examples', 'other_examples', 'demo_tuning_hyperparams.ipynb', 'README.md', '.git']


In [ ]:
import torch
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from IPython.core.debugger import set_trace

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('pytorch version:', torch.__version__)
#print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.10.0+cu111


In [ ]:
# Read in the cleaned data
data_prefix = 'https://raw.githubusercontent.com/nasa-petal/search-engine/main/data/'
df = pd.read_csv(data_prefix + 'cleaned.csv')

# convert array strings to actual arrays
from ast import literal_eval
df['y'] = df['y'].apply(literal_eval)
df.head()

,y,text
0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]",building a home from foam tungara frog foam ne...
1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",a nocturnal mammal the greater mouse eared bat...
2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",polarization sensitivity in two species of cut...
3,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]",identification and characterization of a multi...
4,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",differences in polysaccharide structure betwee...


In [ ]:
from bert_sklearn import BertClassifier
from sklearn.model_selection import cross_validate
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from tempfile import mkdtemp

model = BertClassifier('scibert-scivocab-uncased',
                            max_seq_length=64, #178, #512 max
                            epochs=3, #3 or 4
                            learning_rate=1e-05, #3e-5,
                            train_batch_size=16,
                            eval_batch_size=16,
                            gradient_accumulation_steps=4)

ml_model = OneVsRestClassifier(model)
#ml_model = MultiOutputClassifier(model) # AttributeError: 'BertClassifier' object has no attribute 'classes_'



Building sklearn text classifier...


In [ ]:
cv = MultilabelStratifiedKFold(n_splits=4, shuffle=True, random_state=42) # 8 models per split with multioutputclassifier, 10 per split with onevsrest.
# training the last model in both splits fails at 3% in both multioutput and onevsrest.
# RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 11.17 GiB total capacity; 10.41 GiB already allocated; 5.81 MiB free; 10.65 GiB reserved in total by PyTorch)
# If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
nested_score = cross_validate(ml_model, X=df["text"], y=df["y"].tolist(), scoring="average_precision", cv=cv)
print(f"The mean MAP score using nested cross-validation is: "
      f"{nested_score['test_score'].mean():.3f} +/- {nested_score['test_score'].std():.3f}")

Building sklearn text classifier...
Building sklearn text classifier...


100%|██████████| 410593280/410593280 [00:12<00:00, 33972368.81B/s]


Loading scibert-scivocab-uncased model...


100%|██████████| 410593280/410593280 [00:16<00:00, 25133638.46B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 479, validation data size: 53



...

Training  :   0%|          | 0/120 [00:00<?, ?it/s]

Validating:   0%|          | 0/4 [00:00<?, ?it/s]


Epoch 1, Train loss: 0.3515, Val loss: 0.3120, Val accy: 88.68%

Building sklearn text classifier...
Building sklearn text classifier...
Loading scibert-scivocab-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 482, validation data size: 53



Training  :   0%|          | 0/121 [00:00<?, ?it/s]

The mean MAP score using nested cross-validation is: nan +/- nan


In [ ]:
train_features = pd.read_csv(data_prefix + 'train_features.csv')
train_labels = pd.read_csv(data_prefix + 'train_labels.csv')
test_features = pd.read_csv(data_prefix + 'test_features.csv')
test_labels = pd.read_csv(data_prefix + 'test_labels.csv')

# convert array strings to actual arrays
from ast import literal_eval
train_labels['y'] = train_labels['y'].apply(literal_eval)
test_labels['y'] = test_labels['y'].apply(literal_eval)

X_train = train_features['text']
y_train = train_labels['y']
X_test = test_features['text']
y_test = test_labels['y']

ml_model.fit(X_train, y_train.tolist())

Building sklearn text classifier...
Loading scibert-scivocab-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 576, validation data size: 64



Training  :   0%|          | 0/144 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
from sklearn.metrics import average_precision_score, classification_report, multilabel_confusion_matrix

In [ ]:
# score model
#accy = ml_model.score(X_test, y_test.tolist())

# make class probability predictions
#y_prob = ml_model.predict_proba(X_test)
#print("class prob estimates:\n", y_prob)

# make predictions
y_pred = ml_model.predict_proba(X_test)
'''
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred, y_test.tolist()) * 100))

target_names = ['negative', 'positive']
print(classification_report(y_test, y_pred, target_names=target_names))
'''

Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

...

Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

Predicting:   0%|          | 0/14 [00:00<?, ?it/s]

'\nprint("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred, y_test.tolist()) * 100))\n\ntarget_names = [\'negative\', \'positive\']\nprint(classification_report(y_test, y_pred, target_names=target_names))\n'

In [ ]:
average_precision_score(y_test.tolist(), y_pred) #.38

ValueError: ignored

In [ ]:
multilabel_confusion_matrix(y_test.tolist(), y_pred)

ValueError: ignored